In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
import tkinter as tk
from tkinter import filedialog
import time



In [2]:
# Define class labels
class_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']



In [3]:
# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# Preprocess the data
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)



In [4]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])



In [5]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [6]:
# Train the model
start_time = time.time()
model.fit(train_images, train_labels, epochs=15, batch_size=64, validation_split=0.2)
train_time = time.time() - start_time
print("Training time: ", train_time)




Epoch 1/15
750/750 [==============================] - 46s 58ms/step - loss: 0.5915 - accuracy: 0.7828 - val_loss: 0.3963 - val_accuracy: 0.8550
Epoch 2/15
750/750 [==============================] - 42s 56ms/step - loss: 0.3652 - accuracy: 0.8669 - val_loss: 0.3410 - val_accuracy: 0.8751
Epoch 3/15
750/750 [==============================] - 43s 57ms/step - loss: 0.3144 - accuracy: 0.8854 - val_loss: 0.3147 - val_accuracy: 0.8855
Epoch 4/15
750/750 [==============================] - 41s 54ms/step - loss: 0.2857 - accuracy: 0.8952 - val_loss: 0.2887 - val_accuracy: 0.8937
Epoch 5/15
750/750 [==============================] - 41s 55ms/step - loss: 0.2612 - accuracy: 0.9031 - val_loss: 0.3048 - val_accuracy: 0.8873
Epoch 6/15
750/750 [==============================] - 46s 61ms/step - loss: 0.2399 - accuracy: 0.9114 - val_loss: 0.2708 - val_accuracy: 0.9028
Epoch 7/15
750/750 [==============================] - 50s 67ms/step - loss: 0.2233 - accuracy: 0.9176 - val_loss: 0.2652 - val_accuracy:

In [7]:
def preprocess_and_predict_image(image_path):
    # Read and preprocess the image
    img = preprocess_image(image_path)
    # Predict the class and measure prediction time
    start_time = time.time()
    predictions = model.predict(img)
    pred_time = time.time() - start_time
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_labels[predicted_class_index]  # Get the class name from class_labels
    return predicted_class, pred_time


def preprocess_image(image_path):
    # Load the image from the file path
    img = load_img(image_path, target_size=(28, 28), color_mode="grayscale")
    # Convert the image to a NumPy array
    img_array = img_to_array(img)
    # Invert the grayscale values (black -> white, white -> black)
    img_array = 255 - img_array
    # Reshape the image array to add a dimension for the color channel (even though it's grayscale)
    img_array = img_array.reshape((1, 28, 28, 1))
    # Normalize the pixel values to be between 0 and 1
    return img_array / 255.0


def browse_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        predicted_class, pred_time = preprocess_and_predict_image(file_path)
        result_label.config(text=f"Predicted Class: {predicted_class}\nPrediction Time: {pred_time:.4f} seconds")




In [8]:
# Create a Tkinter window
root = tk.Tk()
root.title("Fashion MNIST Image Classifier")

# Create a button to browse the image
browse_button = tk.Button(root, text="Browse Image", command=browse_image)
browse_button.pack(pady=10)

# Label to display the predicted class
result_label = tk.Label(root, text="")
result_label.pack(pady=5)

# Start the Tkinter event loop
root.mainloop()
